<a href="https://colab.research.google.com/github/Mohammed-Abdul-Rafe-Sajid/Alzheimer-MRI-Disease-Classification-Deep-Learning/blob/main/final_alzheimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from PIL import Image
import io
import matplotlib.pyplot as plt
!pip install torch torchvision
# Loading the datasets



In [ ]:
train_df = pd.read_parquet('/content/drive/MyDrive/CBIT/Data/test-00000-of-00001-44110b9df98c5585.parquet')
test_df = pd.read_parquet('/content/drive/MyDrive/CBIT/Data/test-00000-of-00001-44110b9df98c5585.parquet')

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # resize image to 224x224
    transforms.ToTensor(),           # convert to PyTorch tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # normalize
])

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class AlzheimerMRIDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_bytes = self.df.iloc[idx]['image']['bytes']
        label = self.df.iloc[idx]['label']
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class AlzheimerMRIDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_bytes = self.df.iloc[idx]['image']['bytes']
        label = self.df.iloc[idx]['label']
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:

from torch.utils.data import DataLoader


# Create dataset objects
train_dataset = AlzheimerMRIDataset(train_df, transform=transform)
test_dataset = AlzheimerMRIDataset(test_df, transform=transform)

# Create DataLoader objects for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)   # Input: 3 channels (RGB), Output: 16 feature maps
        self.pool = nn.MaxPool2d(2, 2)                            # Downsample by 2x
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 28 * 28, 128)                   # Flattened layer after convs
        self.fc2 = nn.Linear(128, 4)                              # 4 classes for Alzheimer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + Pool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pool
        x = self.pool(F.relu(self.conv3(x)))  # Conv3 + ReLU + Pool

        x = x.view(-1, 64 * 28 * 28)          # Flatten before FC
        x = F.relu(self.fc1(x))               # FC1 + ReLU
        x = self.fc2(x)                       # Output layer (logits)
        return x

In [ ]:
model = SimpleCNN()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_accuracy = []
train_loss = []

num_epochs = 5  # or whatever value you are using

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total

    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch [1/5], Loss: 1.0929, Accuracy: 49.69%
Epoch [2/5], Loss: 0.9772, Accuracy: 53.12%
Epoch [3/5], Loss: 0.8930, Accuracy: 57.66%
Epoch [4/5], Loss: 0.8003, Accuracy: 64.69%
Epoch [5/5], Loss: 0.5759, Accuracy: 76.33%


In [ ]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 88.36%


In [ ]:
torch.save(model.state_dict(), "final_alz_model.pth")

In [ ]:
from google.colab import files
files.download("final_alz_model.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>